Identify connections between users in a network. 

In [1]:
# List of users each represented by a dict
users = [
    {"id": 0, "name": "Hero"},
    {"id": 1, "name": "Dunn"},
    {"id": 2, "name": "Sue"},
    {"id": 3, "name": "Chi"},
    {"id": 4, "name": "Thor"},
    {"id": 5, "name": "Clive"},
    {"id": 6, "name": "Hicks"},
    {"id": 7, "name": "Devin"},
    {"id": 8, "name": "Kate"},
    {"id": 9, "name": "Klein"}
]

In [2]:
# Friendships represented as a list of ID pairs / tuples
friendships = [(0, 1), (0, 2), (1, 2), (1, 3), (2, 3), (3, 4),
               (4, 5), (5, 6), (5, 7), (6, 8), (7, 8), (8, 9)]

In [3]:
for user in users:
    user["friends"] = []
for i, j in friendships:
    # works because users[i] is the user whose id is i
    users[i]["friends"].append(users[j]) # add i as a friend of j
    users[j]["friends"].append(users[i]) # add j as a friend of i

Now that each user dict contains that users friends, we can analyze their connetions. 

How many total connections are there in this network?
What is the average number of connections in this network? (are these people social?)

In [8]:
# How many friens does this user have?
def number_of_friends(user):
    """how many friends does _user_ have?"""
    return len(user["friends"]) # length of friend_ids list

# All connections in the "network"
total_connections = sum(number_of_friends(user)
                       for user in users)

from __future__ import division # fix int division

num_users = len(users) 
avg_connections = total_connections / num_users # 2.4

print "This group of %s users has an average of %s connections per user." % (num_users, avg_connections)

This group of 10 users has an average of 2.4 connections per user.


Who are the most connected people in this network?

In [9]:
# Create a list of user_id and number_of_friends
num_friends_by_id = [(user['id'], number_of_friends(user))
                    for user in users]

sorted(num_friends_by_id, # sort the list
      key = lambda (user_id, num_friends,): num_friends, # sort it by the num_friends
      reverse=True) # sort from largest to smallest

[(1, 3),
 (2, 3),
 (3, 3),
 (5, 3),
 (8, 3),
 (0, 2),
 (4, 2),
 (6, 2),
 (7, 2),
 (9, 1)]

Each of these pairs is in the format of (user_id, num_friends)
This is basically the network metric degree cetrality. 

A Simple "Friends You May Know" "Suggester" 

In [11]:
def friends_of_friend_ids_bad(user):
    # foaf is abbreviation of friend of a friend
    return [foaf["id"]
           for friend in user["friends"] # for each of the user's friends
           for foaf in friend["friends"]] # get each of their friends

friends_of_friend_ids_bad(users[0])

[0, 2, 3, 0, 1, 3]

"Hero" or users[0] is friends with both of his friends1 and 2. User 3 shows twice since users 3 is accesible as a friend through two different relations: user 1 and user 2 are friends with user 3 and user 0 is friends with both 1 and 2. 

Code Explanation of these relationships:

In [13]:
print [friend['id'] for friend in users[0]['friends']]
print [friend['id'] for friend in users[1]['friends']]
print [friend['id'] for friend in users[2]['friends']]

[1, 2]
[0, 2, 3]
[0, 1, 3]


Implement a count of mutal friends

In [14]:
from collections import Counter

def not_the_same(user, other_user):
    '''two users are not the same if they ahave different ids'''
    return user['id'] != other_user['id']

def not_friends(user, other_user):
    '''other_user is not a friend if he's not in user['friends'];
    if he is not_the_same as all the peple in user['friends']'''
    return all(not_the_same(friend, other_user)
               for friend in user['friends'])

def friends_of_friend_ids(user):
    return Counter(foaf['id']
                   for friend in user['friends'] # for each of my friends
                   for foaf in friend['friends'] # count their friends
                   if not_the_same(user, foaf) # who aren't me
                   and not_friends(user, foaf)) # and aren't my friends

print friends_of_friend_ids(users[3])   

Counter({0: 2, 5: 1})


This tells that the user Chi with id 3 has 2 mutual friends with user Hero with id 0 but only 1 mutual friend with user Clive id 5.

Generate some interests for the users in the form (user id, interst) and put them in a list

In [16]:
interests = [
    (0, "Hadoop"), (0, "Big Data"), (0, "HBase"), (0, "Java"),
    (0, "Spark"), (0, "Storm"), (0, "Cassandra"), 
    (1, "NoSQL"), (1, "MongoDB"), (1, "Cassandra"), (1, "HBase"),
    (1, "Postgres"), (1, "Python"), (2, "scikit-learn"), (2, "scipy"),
    (2, "scikit-learn"),
]